In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings

warnings.filterwarnings('ignore')

# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

In [17]:
df = pd.read_csv('Data/StudentsPerformance.csv')

In [18]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Here we take Math_score as our target feature


In [19]:
X = df.drop(columns=['math score'], axis=1)
y = df['math score']

In [20]:
num_feature = X.select_dtypes(exclude="object").columns
cat_feature = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numarical_transfrom = StandardScaler()
categorical_transfrom = OneHotEncoder()

preprocessor = ColumnTransformer(
    
          transformers=[
        ("OneHotEncoder", categorical_transfrom, cat_feature),
        ("StandardScaler", numarical_transfrom, num_feature)
    ]
    
)

In [21]:
from sklearn.model_selection import train_test_split 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=30, random_state=42)

In [23]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [26]:
def measure_the_error(original, predict):
    mse = mean_squared_error(original, predict)
    mae = mean_absolute_error(original, predict)
    rmse = np.sqrt(mean_squared_error(original, predict))
    r2_sco = r2_score(original, predict)
    return mse , mae, rmse, r2_sco
    

In [31]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #mesure for train data
    train_mse , train_mae, train_rmse, train_r2_sco = measure_the_error(y_train,y_train_pred)
    #measure for test data
    test_mse , test_mae, test_rmse, test_r2_sco = measure_the_error(y_test,y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model performance for train data")
    print("- Mean Squred Error: {:.4f}".format(train_mse))
    print("- Mean Absolute Error: {:.4f}".format(train_mae))
    print("- Root Mean Squred Error: {:.4f}".format(train_rmse))
    print("- R2_Score: {:.4f}".format(train_r2_sco))
    
    print ("------------------------------")
    
    print("Model performance for test data")
    print("- Mean Squred Error: {:.4f}".format(test_mse))
    print("- Mean Absolute Error: {:.4f}".format(test_mae))
    print("- Root Mean Squred Error: {:.4f}".format(test_rmse))
    print("- R2_Score: {:.4f}".format(test_r2_sco))
    
    
    r2_list.append(test_r2_sco)
    
    print("="* 35)
    print('\n')

  
    
    


Linear Regression
Model performance for train data
- Mean Squred Error: 28.2842
- Mean Absolute Error: 4.2467
- Root Mean Squred Error: 5.3183
- R2_Score: 0.8751
------------------------------
Model performance for test data
- Mean Squred Error: 32.3801
- Mean Absolute Error: 4.3786
- Root Mean Squred Error: 5.6903
- R2_Score: 0.8945


Lasso
Model performance for train data
- Mean Squred Error: 43.0537
- Mean Absolute Error: 5.1787
- Root Mean Squred Error: 6.5615
- R2_Score: 0.8099
------------------------------
Model performance for test data
- Mean Squred Error: 55.5974
- Mean Absolute Error: 6.0413
- Root Mean Squred Error: 7.4564
- R2_Score: 0.8189


Ridge
Model performance for train data
- Mean Squred Error: 28.2179
- Mean Absolute Error: 4.2380
- Root Mean Squred Error: 5.3121
- R2_Score: 0.8754
------------------------------
Model performance for test data
- Mean Squred Error: 32.1131
- Mean Absolute Error: 4.3738
- Root Mean Squred Error: 5.6668
- R2_Score: 0.8954


K-Neighbor

In [32]:
pd.DataFrame(
    list(zip(model_list, r2_list)),
    columns=['Model Name', 'R2_Score']
).sort_values(by=['R2_Score'], ascending=False)

,Model Name,R2_Score
2,Ridge,0.895385
0,Linear Regression,0.894516
7,AdaBoost Regressor,0.877125
5,Random Forest Regressor,0.876005
6,XGBRegressor,0.837313
1,Lasso,0.818881
4,Decision Tree,0.787055
3,K-Neighbors Regressor,0.781265


In [13]:
lin_model = LinearRegression(fit_intercept=True)
lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
r2_score = r2_score(y_test,y_pred)
print(r2_score)

0.8945156773295457
